In [2]:
import cudf
import cupy as cp

print("✅ cuDF version:", cudf.__version__)

# Comprobar GPU
props = cp.cuda.runtime.getDeviceProperties(0)
gpu_name = props["name"].decode("utf-8") if isinstance(props["name"], bytes) else props["name"]
print("✅ GPU detectada por CuPy:", gpu_name)

# DataFrame simple en GPU
gdf = cudf.DataFrame({"a": [1, 2, 3, 4], "b": [10, 20, 30, 40]})

print("\n📊 DataFrame cuDF:")
print(gdf)

# Operaciones básicas
gdf["c"] = gdf["a"] + gdf["b"]
print("\n📊 DataFrame con nueva columna (a + b):")
print(gdf)

print("\n✅ Test cuDF básico completado.")


✅ cuDF version: 25.10.00
✅ GPU detectada por CuPy: NVIDIA GeForce RTX 5090

📊 DataFrame cuDF:
   a   b
0  1  10
1  2  20
2  3  30
3  4  40

📊 DataFrame con nueva columna (a + b):
   a   b   c
0  1  10  11
1  2  20  22
2  3  30  33
3  4  40  44

✅ Test cuDF básico completado.


In [3]:
import cudf
from cuml.cluster import DBSCAN

# Dataset con dos grupos y algo de ruido
gdf = cudf.DataFrame(
    {
        "x": [1, 2, 1.5, 8, 9, 8.5, 50],
        "y": [1, 2, 1.2, 8, 9, 8.3, -10],
    }
)

print("📊 Datos de entrada:")
print(gdf)

dbscan = DBSCAN(eps=1.5, min_samples=2)
labels = dbscan.fit_predict(gdf)

print("\n🏷️ Labels DBSCAN (cluster -1 suele ser ruido):")
print(labels)

print("\n✅ Test DBSCAN cuML completado.")


📊 Datos de entrada:
      x     y
0   1.0   1.0
1   2.0   2.0
2   1.5   1.2
3   8.0   8.0
4   9.0   9.0
5   8.5   8.3
6  50.0 -10.0

🏷️ Labels DBSCAN (cluster -1 suele ser ruido):
0    0
1    0
2    0
3    1
4    1
5    1
6   -1
dtype: int32

✅ Test DBSCAN cuML completado.


In [4]:
import cudf
from cuml.cluster import KMeans

# Dataset simple con dos grupos claros
gdf = cudf.DataFrame(
    {
        "x": [1, 2, 1.5, 8, 9, 8.5],
        "y": [1, 2, 1.2, 8, 9, 8.3],
    }
)

print("📊 Datos de entrada:")
print(gdf)

# Modelo K-Means en GPU
kmeans = KMeans(n_clusters=2, max_iter=100, random_state=42)
labels = kmeans.fit_predict(gdf)

print("\n🏷️ Labels asignados por K-Means (GPU):")
print(labels)

print("\n📌 Centroides:")
print(kmeans.cluster_centers_)

print("\n✅ Test K-Means cuML completado.")


📊 Datos de entrada:
     x    y
0  1.0  1.0
1  2.0  2.0
2  1.5  1.2
3  8.0  8.0
4  9.0  9.0
5  8.5  8.3

🏷️ Labels asignados por K-Means (GPU):
0    1
1    1
2    1
3    0
4    0
5    0
dtype: int32

📌 Centroides:
     0         1
0  8.5  8.433333
1  1.5  1.400000

✅ Test K-Means cuML completado.


In [7]:
import cudf
import cupy as cp
from cuml.cluster import KMeans
from cuml.decomposition import PCA

# Simulamos datos con 10 dimensiones en GPU
n_samples = 5000
n_features = 10
cp.random.seed(42)

data_cp = cp.random.randn(n_samples, n_features).astype(cp.float32)

# Convertimos a cuDF DataFrame
gdf = cudf.DataFrame(data_cp, columns=[f"f{i}" for i in range(n_features)])

print("📊 Shape del DataFrame:", gdf.shape)

# PCA a 2 componentes (sin random_state)
pca = PCA(n_components=2)
gdf_pca = pca.fit_transform(gdf)

print("\n📊 Primeras filas después de PCA (2D):")
print(gdf_pca.head())

# K-Means sobre datos reducidos
kmeans = KMeans(n_clusters=3, max_iter=200, random_state=42)
labels = kmeans.fit_predict(gdf_pca)

print("\n🏷️ Distribución de clusters (conteo por label):")
print(labels.value_counts())

print("\n✅ Test PCA + KMeans cuML completado.")

📊 Shape del DataFrame: (5000, 10)

📊 Primeras filas después de PCA (2D):
          0         1
0 -0.068215 -0.968453
1 -0.137617 -0.313526
2 -1.730349  0.616466
3 -0.130284  0.114181
4  0.525167  0.544509

🏷️ Distribución de clusters (conteo por label):
0    1696
2    1686
1    1618
Name: count, dtype: int64

✅ Test PCA + KMeans cuML completado.
